# 📊 Nursing FunctionGemma Evaluation v2

This notebook evaluates the **improved v2 model** trained on diverse examples.

**Expected Improvement:**
- Model should now extract actual VALUES (120, 80, 72)
- Not just output function signatures

## Step 1: Mount Drive & Install

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install -q -U transformers peft torch accelerate bitsandbytes huggingface_hub pandas matplotlib seaborn tqdm

## Step 2: Login to Hugging Face

In [ ]:
from huggingface_hub import login
login()

## Step 3: Load Evaluation Dataset

In [ ]:
import json
import os

EVAL_PATH = "/content/drive/MyDrive/nmc_brain/data/function_eval_dataset.json"

with open(EVAL_PATH, 'r') as f:
    eval_data = json.load(f)

print(f"✅ Loaded {len(eval_data)} evaluation examples")

## Step 4: Load v2 Model

In [ ]:
import torch
import torch.distributed as dist
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

# Fix distributed
try:
    if not dist.is_initialized():
        dist.init_process_group(backend="gloo", init_method="file:///tmp/eval_init", rank=0, world_size=1)
except:
    pass

BASE_MODEL_ID = "google/medgemma-4b-it"
ADAPTER_ID = "NurseCitizenDeveloper/nursing-function-gemma"

print("Loading base model...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    quantization_config=bnb_config,
    device_map={"": 0},
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

print(f"Loading adapter: {ADAPTER_ID}")
model = PeftModel.from_pretrained(model, ADAPTER_ID)
print("✅ v2 Model loaded!")

## Step 5: Define Prediction Function

In [ ]:
import re

def generate_prediction(user_input):
    """Generate a function call prediction using v2 prompt format."""
    # Use same prompt format as training
    tools_prompt = """You are a clinical AI agent. Convert clinical notes into function calls.

Functions:
- record_vitals(systolic=X, diastolic=Y, heart_rate=Z, temp_c=T)
- administer_medication(drug_name='X', dose='Y', route='Z')
- search_nmc_standards(query='X')

Extract the actual values from the input and output the correct function call."""

    prompt = f"<start_of_turn>user\n{tools_prompt}\n\nInput: {user_input}<end_of_turn>\n<start_of_turn>model\n"
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            do_sample=False,
            eos_token_id=tokenizer.eos_token_id
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    if "<start_of_turn>model" in response:
        response = response.split("<start_of_turn>model")[-1].strip()
    if "<end_of_turn>" in response:
        response = response.split("<end_of_turn>")[0].strip()
    
    # Extract function call
    match = re.search(r'<function_call>(.*?)</function_call>', response, re.DOTALL)
    if match:
        return match.group(1).strip()
    
    # Try raw function call
    for func_name in ['record_vitals', 'administer_medication', 'search_nmc_standards']:
        if func_name in response:
            match = re.search(rf'{func_name}\([^)]+\)', response)
            if match:
                return match.group(0)
    
    return response

# Quick test
test_result = generate_prediction("BP is 120/80, pulse 72")
print(f"Quick test result: {test_result}")

## Step 6: Run Full Evaluation

In [ ]:
import pandas as pd
from tqdm import tqdm

print("Running evaluation on 50 test cases...")
results = []

for item in tqdm(eval_data):
    user_input = item['input']
    expected = item['expected']
    predicted = generate_prediction(user_input)
    
    # Determine function type
    if 'record_vitals' in expected:
        func_type = 'vitals'
    elif 'administer_medication' in expected:
        func_type = 'medication'
    else:
        func_type = 'nmc_search'
    
    # Function name match
    func_name_expected = expected.split('(')[0]
    func_name_correct = func_name_expected in predicted
    
    # Exact match (normalized - remove spaces)
    expected_norm = re.sub(r'\s+', '', expected.lower())
    predicted_norm = re.sub(r'\s+', '', predicted.lower())
    exact_match = expected_norm == predicted_norm
    
    # Value extraction check (has actual numbers, not just param names)
    has_values = bool(re.search(r'=\d+', predicted) or re.search(r"='[^']+'", predicted))
    
    results.append({
        'input': user_input,
        'expected': expected,
        'predicted': predicted,
        'func_type': func_type,
        'func_name_correct': func_name_correct,
        'exact_match': exact_match,
        'has_values': has_values
    })

df = pd.DataFrame(results)
print(f"\n✅ Evaluation complete! {len(df)} examples processed.")

## Step 7: Results Summary

In [ ]:
overall_accuracy = df['exact_match'].mean() * 100
func_name_accuracy = df['func_name_correct'].mean() * 100
value_extraction = df['has_values'].mean() * 100

print("="*50)
print("📊 v2 MODEL RESULTS")
print("="*50)
print(f"\n🎯 Exact Match Accuracy: {overall_accuracy:.1f}%")
print(f"🔧 Function Name Accuracy: {func_name_accuracy:.1f}%")
print(f"📝 Value Extraction Rate: {value_extraction:.1f}%")
print(f"📊 Total Test Cases: {len(df)}")

print("\n" + "="*50)
print("📊 ACCURACY BY FUNCTION TYPE")
print("="*50)

for func_type in ['vitals', 'medication', 'nmc_search']:
    subset = df[df['func_type'] == func_type]
    if len(subset) > 0:
        acc = subset['exact_match'].mean() * 100
        func_acc = subset['func_name_correct'].mean() * 100
        val_rate = subset['has_values'].mean() * 100
        print(f"\n{func_type.upper()}:")
        print(f"  - Count: {len(subset)}")
        print(f"  - Exact Match: {acc:.1f}%")
        print(f"  - Function Name Match: {func_acc:.1f}%")
        print(f"  - Value Extraction: {val_rate:.1f}%")

## Step 8: Sample Predictions

In [ ]:
print("\n" + "="*50)
print("📋 SAMPLE PREDICTIONS")
print("="*50)

for i, row in df.head(10).iterrows():
    status = "✅" if row['exact_match'] else "❌"
    print(f"\n{status} Example {i+1}:")
    print(f"   Input: {row['input'][:60]}...")
    print(f"   Expected: {row['expected']}")
    print(f"   Predicted: {row['predicted']}")

## Step 9: Summary for Article

In [ ]:
print("\n" + "="*60)
print("📋 SUMMARY STATISTICS FOR ARTICLE")
print("="*60)
print(f"""
The Nursing FunctionGemma v2 model was evaluated on a held-out test set
of {len(df)} clinical scenarios across three function types.

OVERALL PERFORMANCE:
- Exact Match Accuracy: {overall_accuracy:.1f}%
- Function Name Accuracy: {func_name_accuracy:.1f}%
- Value Extraction Rate: {value_extraction:.1f}%

PERFORMANCE BY FUNCTION TYPE:""")

for func_type in ['vitals', 'medication', 'nmc_search']:
    subset = df[df['func_type'] == func_type]
    if len(subset) > 0:
        acc = subset['exact_match'].mean() * 100
        print(f"- {func_type.title()}: {acc:.1f}% (n={len(subset)})")

print(f"""
TRAINING DETAILS (v2):
- Base Model: google/medgemma-4b-it
- Fine-tuning Method: QLoRA (4-bit quantization)
- Training Examples: 550
- Epochs: 8
- LoRA Rank: 32
- Final Training Loss: 0.077
""")